<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# NLP Using the Twitter API: Guided Lab

_Authors: Dave Yerrington (SF)_

---


<img src="https://snag.gy/RNAEgP.jpg" width="600">

### Can We Correctly Identify Which of These Men Tweeted What?

> *Note: This lab is intended to be a guided lab until the independent practice questions.*


## Goals
---

We're going to attempt to classify whether a tweet comes from Donald Trump or Bernie Sanders. This lab involves multiple steps:
- Create a developer account on Twitter.
- Create a method to pull a list of tweets from the Twitter API.
- Perform proper preprocessing on our text.
- Engineer sentiment features in our data set using `TextBlob`.
- Explore supervised classification techniques.

## Twitter API Developer Registration
---

If you haven't registered for a Twitter account, do so now in order to have a developer account for this lab.

[Twitter Rest API](https://dev.twitter.com/rest/public)


## Create an App

---

![](https://snag.gy/HPBQbJ.jpg)

Go to Twitter and register an app: [apps.twitter.com](https://apps.twitter.com/).

> **Note**: For the required website field, you can put a placeholder.

After you set up your application, you’ll only need to reference the corresponding keys Twitter generates for it. These are the keys that we'll use with our application to communicate with the Twitter API.

## Install Python Twitter API Library

---

Someone was nice enough to build a Python library for us, which makes pulling tweets simple: We only need to plug in our keys to start collecting data. The library we'll be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, just run the next frame (there is no conda package).

In [ ]:
# NOTE: do NOT install the library "twitter" (with pip), 
# it's a different library and has a different interface!

#!pip install twitter python-twitter

## Some Twitter Rules
---

**Twitter notifies you that your requests have a rate limit.**

> When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate-limit window, then it allows you to make 15 requests per window on behalf of your application. This limit is considered separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of Twitter's rules:

![](https://snag.gy/yJ6vIH.jpg)


## About Those Keys: OAuth Review
---

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's Going On Here?  

## Our Application Keys
---

Take note of the application keys you’ll use to connect to Twitter and mine tweets from the official Bernie Sanders and Donald Trump accounts.

![](https://snag.gy/H1djQK.jpg)


## `TweetMiner` Class Structure

---

The following code will get you up and running by providing connectivity to Twitter. The class has the ability to make requests and can eventually transform the JSON responses into DataFrames.

This is a great example of using object-oriented Python to organize our code.

> **Note:** `request_limit` is used in this class to limit the number of tweets that are pulled per instance request. Setting it to something lower until you've worked out the bugs in your request and captured the data you want is essential for avoiding rate limit blocks.

### Twitter API Key Set Up

Fill the information below in with the keys for your account.

- **consumer_key**: Find this on your application page under the Keys and Access Tokens tab.
- **consumer_secret**: This is located right under **consumer_key** in the Keys and Access Tokens tab.
- **access_token_key**: To get this, you'll need to click the button to generate tokens.
- **access_token_secret**: This is available after you generate tokens.

In [ ]:
# !pip uninstall twitter -y
# !pip install python-twitter

# NOTE: do NOT install the library "twitter" (with pip), 
# it's a different library and has a different interface!

In [6]:
import twitter, re, datetime
import pandas as pd

# Your keys go here:
twitter_keys = {
    'consumer_key':        'lA6UQplM5sxIAFr83ueUl9sgE',
    'consumer_secret':     'f94t4BD6Vj7aCVkX1qAIVwsP4x69J2vXvm61FTIuwb9GfmdsuP',
    'access_token_key':    '1203210464-FYE1FvoUx1GjVcoyok3U1brWDpELBJEcSNGC1OC',
    'access_token_secret': 'kuVli1j010RPdYTbozFiIyp8QxX6PqRApdi49baLqBgzo'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)


In [7]:
class TweetMiner(object):

    result_limit    =   20    
    api             =   False
    data            =   []
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.api = api
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dyerrington", mine_rewteets=False, max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'handle':          item.user.name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

## Instantiate the Class
---

Make sure you pass the keys dictionary and the API as arguments.

**Check:** Call the object's `mine_user_tweets()` method, providing a user from whom to pull tweets.

In [ ]:
# A:

### Convert the Tweet Outputs to a Pandas DataFrame

> *Hint: This is as easy as passing it to the DataFrame constructor!*

In [ ]:
# A:

##  Create the Training Data

---

Let's get our mined data from the Twitter API.  

- Mine Trump tweets.
- Create a tweet DataFrame.
- Mine Sanders tweets.
- Append the results to our DataFrame.

In [ ]:
# A:

## Are There Any Interesting N-Grams Going on With Trump?
---

Set up a vectorizer from scikit-learn and fit the text of Trump's tweets with an n-gram range of two to four. Identify the most common n-grams.

> **Note:** It's up to you whether or not you want to remove stop words. How does keeping or removing stop words affect the results?

In [ ]:
# A:

### Look at the N-Grams for Bernie Sanders

In [ ]:
# A:

## Processing the Tweets and Building a Model

---

To perform classification, we'll need to convert the tweets into a set of features.

**You will need to:**
- Vectorize and input text data.
- Initialize a model (try logistic regression).
- Train, predict, and cross-validate.
- Evaluate the performance of the model.

> **Bonus:** You may have noticed that there are website links in the tweets. What additional preprocessing steps can you take before building the model?


In [ ]:
# A:

## Check the Predicted Probability for a Random Sanders and Trump Tweet
---

A couple of tweets from both Sanders and Trump are provided below. 

Estimate the predicted probability of Trump being the author for the two tweets.

In [ ]:
# Prep our source as TF-IDF vectors.
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# Note: Do not re-initialize the TF-IDF vectorizer or the feature space will be overwritten and
# your transform will not match the number of features on which you trained your model.
#
# You only need to transform because you fit previously.
#
####


## Independent Practice Questions

---

### 1) Pull tweets for some new users.

Experiment with using more data. The API will not like it if you blow through its limits, so be careful. Try to grab only what you need one time, then work on the copy of the objects that are returned.  

> Read the documentation about rate limits and see if you can get enough without hitting the limit. Are there any options available in the API to avoid such a problem?

**Pull tweets for more than two different users of your choice.**

In [ ]:
# A:

### 2) Build a multi-class classification model to distinguish between the users.

Try a different model than what we used before.

In [ ]:
# A:

### 3) Create a confusion matrix and a classification report.

In [ ]:
# A:

### 4) What are the most and least "distinctive" tweets for each user?

To find this, identify the tweet that has the highest (correct) predicted probability of being that user's tweet for each user.

In [ ]:
# A: